<a href="https://colab.research.google.com/github/twaldburger/flood475/blob/master/geo475_flood_prediction_in_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geo475 - Flood prediction in Google Earth Engine
Go through the notebook and run the cells. Try to understand the code and the overall approach.  
Feel free to update code where you see room for improvement.  

There are several tasks in the notebook. Please try to solve them but don't spend too much time on them. The goal is to understand the process and the code - solving all the tasks is secondary.  

A notebook including some of countless solutions to the tasks can be found [here](https://github.com/twaldburger/flood475-presenter).


---
## Setup

In [1]:
import ee
import geemap
import geemap.colormaps as cm
import time

## set some parameters, please update with your project id
PROJECT_ID = 'ee-timwaldburger-flood475' # your GEE project id #TODO
SAMPLE_SIZE = 100 # number of training locations per flood event and class
SEED = 3414 # for reproducible results

## connect to GEE
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


---
## Data exploration and visualization

In [2]:
## define the datasets from which to derive the input features
globalFlood = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
        .select('DEM')
landcover = ee.ImageCollection('ESA/WorldCover/v200')
hydro = ee.Image('MERIT/Hydro/v1_0_1')
prec = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
         .select('total_precipitation')
runoffPotential = ee.Image('projects/sat-io/open-datasets/HiHydroSoilv2_0/Hydrologic_Soil_Group_250m') \
                    .remap([1, 2, 3, 4, 14, 24, 34], [1, 2, 3, 4, 1, 3, 4]) \
                    .select('remapped')

> **Task:** Get an overview on the datasets we use. What data do they provide? Who produced them? What is their spatial and temporal resolution? A good starting point is the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets).

Some things to discuss:
- How to find information efficiently?
- What are characteristics of the datasets worth mentioning?
- What are potential problems when using these datasets to train a model?

Datasets:
- [Global Flood Database v1 (2000-2018)](https://developers.google.com/earth-engine/datasets/catalog/GLOBAL_FLOOD_DB_MODIS_EVENTS_V1#description): 913 flood events between 2000-2018 at 30m resolution. Provided by [Dartmouth Flood Observatory ](https://floodobservatory.colorado.edu/) and based on MODIS data.
- [Copernicus DEM GLO-30: Global 30m Digital Elevation Model](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30#description): Global Digital Surface Model at 30m resolution. Provided by [Copernicus/ESA](https://spacedata.copernicus.eu/collections/copernicus-digital-elevation-model) and based on data acquired through the TanDEM-X mission between 2011 and 2015.
- [ESA WorldCover 10m v200](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200?hl=en#description): Global landcover map for 2021 with 11 landcover classes at 10m resolution. Provided by [ESA](https://esa-worldcover.org/en).
- [MERIT Hydro: Global Hydrography Datasets](https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description): Global flood direction map at \~90m resolution (at equator). The dataset is provided by the [University of Tokyo](http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/index.html) and based on various elevation and water body dataset with vintages between 1987 and 2017.
- [ERA5-Land Hourly - ECMWF Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY): Global reanalysis dataset containing hourly climate variables at ~11km resolution. Provided by [Copernicus/ESA](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview).
- [Soil Grids 250m v2.0](https://gee-community-catalog.org/projects/isric/): Global dataset of soil properties at 250m resolution provided by (ISRIC)[https://www.isric.org/].
</font>

In [29]:
## subtract the permanent water bodies from the flooded areas
def subtractPermanentWater(img):
  flood = img.select('flooded')
  perm = img.select('jrc_perm_water')
  return flood.multiply(perm.eq(0))
flood = globalFlood.map(subtractPermanentWater).sum()

## initialize map and add some basemaps
# see https://stackoverflow.com/a/33023651 for Google basemap list
Map = geemap.Map(center=[27, -81], zoom=7, basemap='CartoDB.DarkMatter')
Map.add_basemap('CartoDB.Positron', show=False)
Map.add_tile_layer("https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}", name="Google.Roadmap", attribution="Google", shown=False)
Map.add_tile_layer("https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}", name="Google.Satellite", attribution="Google", shown=False)

## add historic floods
flood_vis = {'min':0, 'max':10, 'palette':cm.palettes.Blues}
Map.add_layer(flood.selfMask(), flood_vis, 'Historic floods')
Map.add_colorbar(flood_vis, label="Number of floods", layer_name="Historic floods")

## add elevation
elevation_vis = {'min':0, 'max':3000, 'palette':cm.palettes.dem}
Map.addLayer(dem.mosaic(), elevation_vis, 'Elevation', shown=False)
Map.add_colorbar(elevation_vis, label="Elevation [m]", layer_name="Elevation")

## add landcover
landcover_vis = {'bands':['Map']}
Map.addLayer(landcover.first(), landcover_vis, 'Landcover', shown=False)
Map.add_legend(title="Landcover", builtin_legend="ESA_WorldCover", layer_name='Landcover')

## add upstream drainage area
upa_vis = {'min':0, 'max':10, 'palette':cm.palettes.Purples}
Map.addLayer(hydro.select('upa'), upa_vis, 'Upstream drainage area', shown=False)
Map.add_colorbar(upa_vis, label="Upstream drainage area [km^2]", layer_name="Upstream drainage area")

## add precipitation
prec_vis = {'min':0, 'max':5, 'palette':cm.palettes.turbo}
Map.addLayer(prec.filter(ee.Filter.date('2024-10-01', '2024-10-31')).sum(), prec_vis, 'Precipitation', shown=False)
Map.add_colorbar(prec_vis, label="Precipitation [m]", layer_name="Precipitation")

## add runoff potential
ee_class_table = """
Value	Color	Description
1	edf8e9	HSG-A: low runoff potential (>90% sand and <10% clay)
2	bae4b3	HSG-B: moderately low runoff potential (50-90% sand and 10-20% clay)
3	74c476	HSG-C: moderately high runoff potential (<50% sand and 20-40% clay)
4	238b45	HSG-D: high runoff potential (<50% sand and >40% clay)
"""
runoff_legend = geemap.legend_from_ee(ee_class_table)
runoff_vis = {'min':0, 'max':4, 'palette':list(runoff_legend.values())}
Map.addLayer(runoffPotential, runoff_vis, 'Runoff potential', shown=False)
Map.add_legend("Runoff potential", legend_dict=runoff_legend, layer_name="Runoff potential")

## display map
Map

Map(center=[27, -81], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

> **Task:** Add the input datasets to the map so you can explore them visually. We are interested in elevation, landcover, upstream drainage area, precipitation and runoff potential. You can find the geemap documentation [here](https://geemap.org/).

Some things to discuss:
- Does anyone want to share their map?
- How to add layers with legends and colorbars in geemap?
- Any new specific findings on data format when visualizing the datasets?
- How are the floods distributed? Are there geographies without any or with only a few floods?

---
## Training dataset



In [6]:
def pointQuery(fc:ee.FeatureCollection, img:ee.Image, prop:str) -> ee.FeatureCollection:
  """
  Returns pixel values at locations using a feature collection and an image.

  Parameters
  ----------
  fc: : ee.FeatureCollection
    Collection of points at which to query the image.
  img : ee.Image
    The image to query.
  prop : str
    Name of new property to hold the query results.

  Returns
  -------
  ee.FeatureCollection
    Input FeatureCollection with lookup values added as new property.
  """
  fc = img.reduceRegions(collection=fc, reducer=ee.Reducer.first())
  return fc.map(lambda feat: feat.set(prop, feat.get('first')))


def removeProperty(fc:ee.FeatureCollection, prop:str) -> ee.FeatureCollection:
  """
  Removes a property by name from a feature collection.

  Parameters
  ----------
  fc : ee.FeatureCollection
    Collection from which to remove the property.
  prop : str
    Property to remove.

  Returns
  -------
  ee.FeatureCollection
    Input collection without the removed property.
  """
  selectProperties = fc.propertyNames().filter(ee.Filter.neq('item', prop))
  return fc.select(selectProperties)


def createSample(img:ee.Image) -> ee.FeatureCollection:
  """
  Samples training dataset for a single flood image.

  Parameters
  ----------
  img : ee.Image
    Input image.

  Returns
  -------
  ee.FeatureCollection
    Sampled and enriched training data.
  """

  ## subtract permanent water bodies from flooded areas
  permanent = img.select('jrc_perm_water')
  water = img.select('flooded')
  flooded = water.subtract(permanent).gt(0)

  ## get the total and maximum precipitation over 14 days prior to the event end date
  end = img.getNumber('system:time_end')
  start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000
  precSum = prec.filter(ee.Filter.date(start, end)).sum()
  precMax = prec.filter(ee.Filter.date(start, end)).max()

  ## sample equal number of flooded and non-flooded points
  sample = flooded.stratifiedSample(numPoints=SAMPLE_SIZE, classBand='flooded', geometries=True)

  ## add image id in case we want to join the event metadata later
  sample = sample.map(lambda x: x.set('eventId', img.get('system:index')))

  ## enrich sample by running point lookups on multiple datasets
  sample = pointQuery(sample, dem.mosaic(), 'demElevationAbs')
  sample = pointQuery(sample, ee.Terrain.aspect(dem.mosaic()), 'demAspect')
  sample = pointQuery(sample, ee.Terrain.slope(dem.mosaic()), 'demSlope')
  sample = pointQuery(sample, landcover.first(), 'landcover')
  sample = pointQuery(sample, hydro.select('upa'), 'upa')
  sample = pointQuery(sample, runoffPotential, 'runoffPot')
  sample = pointQuery(sample, precSum, 'precSum')
  sample = pointQuery(sample, precMax, 'precMax')

  ## remove first-property
  sample = sample.map(lambda feat: removeProperty(feat, 'first'))

  ## normalize elevation
  elevationRange = dem.mosaic().reduceRegion(geometry=img.geometry(), reducer=ee.Reducer.minMax())
  min = ee.Number(elevationRange.get('DEM_min'))
  max = ee.Number(elevationRange.get('DEM_max'))
  def normalizeElevation(feat:ee.Feature) -> ee.Feature:
    return feat.set('demElevationNorm', (ee.Number(feat.get('demElevationAbs')).subtract(min)).divide(max.subtract(min)))
  sample = sample.filter(ee.Filter.notNull(ee.List(['demElevationAbs']))).map(normalizeElevation)

  return sample

> **Task:** Try to understand the code in the cell above. Why are we using *ee.Image.stratifiedSample* in line 70 instead of using the much faster *ee.Image.Sample* method? Why are we using the complicated GEE methods in lines 65-76 and 90-94 instead of simply using plain Python? What does *ee.FeatureCollection.map* do? Why do we not just write a simple for-loop instead? Why do we normalize the elevation values?

Some things to discuss:
- Where can we quickly look up documentation on GEE classes and functions?
- Check out the documentation section in the [GEE Code editor](https://code.earthengine.google.com/) if not done already.  

Answers to questions:
- *stratifiedSample* samples the same number of locations within each class. Since our flood footprints mostly contain non-flooded pixels, using *stratifiedSample* is a convenient way to avoid oversampling non-flooded pixels. However, there might also be a risk of *stratifiedSample* oversampling certain flooded areas if the flood footprint is very small.
- We want to execute all the code in GEE itself so we can benefit from its optimization and parallelisation. If we would use plain Python, we would need to fetch the info from GEE server into our Colab Runtime which would make the whole process extremely inefficient.
- *map* iterates over a feature collection and applies an algorithm to each feature. This process is run in parallel on the GEE server. Using a Python for-loop instead would loose the parallel execution and also create all the problems mentioned in the last question.
- We use a global flood dataset, but look at the individual events independently. We do not want to create a bias towards the general elevation of the region where an event took place and are therefore normalizing the elevation within each image (and therefore for each event).

In [10]:
## create task to enrich sample and store the result as an asset
properties = ['demAspect', 'demElevationAbs', 'demElevationNorm', 'demSlope', 'eventId', 'flooded', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa', '.geo']
sample = globalFlood.map(createSample).flatten()
sample = sample.filter(ee.Filter.notNull(properties)).distinct(properties)
task = ee.batch.Export.table.toAsset(sample, description='flood475-sampling', assetId=f"projects/{PROJECT_ID}/assets/flood475_sample")

In [14]:
## run and monitor the task
task.start()
while task.active():
  ts = task.status()
  if ts['start_timestamp_ms']>0:
    s = round((ts['update_timestamp_ms']-ts['start_timestamp_ms'])/1000)
  else:
    s = round((ts['update_timestamp_ms']-ts['creation_timestamp_ms'])/1000)
  print(f"task '{ts['description']}' is {ts['state']} for {s} seconds")
  time.sleep(60)
task.status()

task 'flood475-sampling' is RUNNING for 0 seconds
task 'flood475-sampling' is RUNNING for 60 seconds
task 'flood475-sampling' is RUNNING for 120 seconds
task 'flood475-sampling' is RUNNING for 180 seconds
task 'flood475-sampling' is RUNNING for 240 seconds
task 'flood475-sampling' is RUNNING for 300 seconds
task 'flood475-sampling' is RUNNING for 360 seconds
task 'flood475-sampling' is RUNNING for 420 seconds
task 'flood475-sampling' is RUNNING for 480 seconds
task 'flood475-sampling' is RUNNING for 540 seconds
task 'flood475-sampling' is RUNNING for 600 seconds
task 'flood475-sampling' is RUNNING for 660 seconds


{'state': 'COMPLETED',
 'description': 'flood475-sampling',
 'priority': 100,
 'creation_timestamp_ms': 1732992304539,
 'update_timestamp_ms': 1732993052658,
 'start_timestamp_ms': 1732992309734,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/ee-timwaldburger-flood475/assets/flood475_sample'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.02915676310658455,
 'id': '2R4EPKA6FQNNMU5J73F437FF',
 'name': 'projects/ee-timwaldburger-flood475/operations/2R4EPKA6FQNNMU5J73F437FF'}

> **Task:** There are multiple places (outside of Google Colab) where we can also monitor our tasks. Can you find them?  

> **Task:** Add the sampled data points to your map from above.

3 places to check tasks:
- [GEE Code editor](https://code.earthengine.google.com/)
- [Task Manager](https://code.earthengine.google.com/tasks)
- [Tasks Page in the Cloud Console](https://console.cloud.google.com/earth-engine/tasks?project=ee-timwaldburger-flood475)

In [30]:
visParams = {
    'color': '#FFFFFF',
    'colorOpacity': 1,
    'pointSize': 5,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 1,
}
Map.add_styled_vector(sample, column='flooded', palette=['#ffffbf', '#d7191c'], layer_name='Training data', **visParams)
Map

Map(bottom=14130.0, center=[27, -81], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

---
## Model training

In [31]:
## import the training dataset
sample = ee.FeatureCollection(f"projects/{PROJECT_ID}/assets/flood475_sample")
#TODO: add my training sample and make accessible for everyone

## partition into 70% training and 30% validation samples
sample = sample.randomColumn('random', seed=SEED)
training = sample.filter(ee.Filter.lt('random', 0.7))
validation = sample.filter(ee.Filter.gte('random', 0.7))

## train a random forest
properties = ['demAspect', 'demElevationNorm', 'demSlope', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa']
randomForest = ee.Classifier.smileRandomForest(10).setOutputMode('CLASSIFICATION')
classifier = randomForest.train(
    features=training,
    classProperty='flooded',
    inputProperties=properties
)

In [33]:
# accuracy on training set
trainConfusionMatrix = classifier.confusionMatrix()
trainFscores = trainConfusionMatrix.fscore().getInfo()
print(f"train accuracy: {trainConfusionMatrix.accuracy().getInfo()}")
print(f"train f-score non-flooded: {trainFscores[0]}")
print(f"train f-score flooded: {trainFscores[1]}")

# accuracy on test set
testConfusionMatrix = validation.classify(classifier).errorMatrix('flooded', 'classification')
testFscores = testConfusionMatrix.fscore().getInfo()
print(f"test accuracy: {testConfusionMatrix.accuracy().getInfo()}")
print(f"test f-score non-flooded: {testFscores[0]}")
print(f"test f-score flooded: {testFscores[1]}")

train accuracy: 0.9156202472409625
train f-score non-flooded: 0.928406581135072
train f-score flooded: 0.8972736875838249
test accuracy: 0.8228139307108854
test f-score non-flooded: 0.8525950770280256
test f-score flooded: 0.7779522164816031


> **Task:** What do accuracy and F1-Score describe? Do you think your model performs well given those results?

Some things to discuss:
- Does the model perform well based on the accuracy and F-Score?

Answers to questions:
- Accuracy: How often is the model right? An accuracy of 0.9 is generally considered good in many machine learning contexts. It means that your model makes the correct prediction 90% of the time.
- F1-Score: Metric that combines precision and recall into a single value. It provides a balanced measure of a model's performance, considering both the accuracy of positive predictions (precision) and the ability to identify all positive instances (recall). An F1-score of 0.8 is generally considered good. It indicates that your model is performing well in terms of both precision and recall.  
- Recall: How many positive predictions can the model identify?
- Precision: How often are positive predictions correct?


---
## Prediction

In [35]:
def predict(roi: ee.Geometry, classifier: ee.Classifier) -> ee.Image:
  """
  Predict flood probability for a given region of interest.

  Parameters
  ----------
  roi : ee.Geometry
    Region of interest.
  classifier : ee.Classifier
    Trained classifier.

  Returns
  -------
  ee.Image
    Flood probabilities.
  """

  ## normalize elevation
  elevationRange = dem.mosaic().reduceRegion(geometry=roi, reducer=ee.Reducer.minMax(), scale=30, bestEffort=True)
  min = ee.Number(elevationRange.get('DEM_min'))
  max = ee.Number(elevationRange.get('DEM_max'))

  ## calculate start and end data for precipitation data aggregation
  end = ee.Date('2024-10-31T00:00:00').millis()
  start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000

  ## create composite of all relevant datasets
  composite = ee.Image.cat(
      ee.Terrain.aspect(dem.mosaic()),
      dem.mosaic().unitScale(min, max),
      ee.Terrain.slope(dem.mosaic()),
      landcover.first(),
      prec.filter(ee.Filter.date(start, end)).max(),
      prec.filter(ee.Filter.date(start, end)).sum(),
      runoffPotential,
      hydro.select('upa')
  ).rename(properties)

  ## classify the composite
  classifier = classifier.setOutputMode('MULTIPROBABILITY')
  classified = composite.classify(classifier).clip(roi)
  probabilities = classified.arrayFlatten([['non-flooded', 'flooded']])
  probability = probabilities.select('flooded')

  return probability

> **Task:** Try to understand the code in the cell above. What does the *predict*-function do?

Answer to the question:
- The *predict*-function creates a composite image by combining all the datasets we used for model training. It normalizes the elevation and aggregates precipitation. It then classifies every pixel in the composite using the pre-trained model.

In [36]:
## run prediction for current map bounds
probability = predict(roi=ee.Geometry.BBox(*Map.getBounds()), classifier=classifier)

## update mask to exclude permanent water
permanentWater = globalFlood.select('jrc_perm_water').mosaic()
probability = probability.updateMask(permanentWater.neq(1))

## update the map
probability_vis = {'min':0, 'max':1, 'palette':cm.palettes.cividis}
Map.addLayer(probability.selfMask(), probability_vis, 'Flooded probability')
Map.add_colorbar(probability_vis, label="Flooded probability", layer_name="Flooded probability", font_size=9)
Map

Map(bottom=3879.0, center=[22.14670778001263, -66.01762052261185], controls=(WidgetControl(options=['position'…

> **Task:** Remove pixels representing permanent water bodies from the prediction.  

> **Task:** Check various areas on the map. Where does the model seem to perform well. Where does it perform bad? Can you explain why it performs bad in certain areas? Do you see any bias towards a specific feature?

> **Task:** Go back to the cell where we trained the model and play around with hyperparameters. Can you improve the model?  

> **Task:** Check in the GEE documentation which other models are available and try to run them. What differences do you see? Which model works best?

Some things to discuss:
- The model seems to assign a high flood probability to mountain ranges and and glaciers. This does not make much sense. A possible explanation is the lack of such areas in the training data.

---
## Bonus: run country-scale prediction
The code below runs the trained model for all of Switzerland at 30 m resolution and exports the result to GEE. It runs in about 20 minutes. No need to run in the lab, but feel free to try it out and play around with different models, geographies and settings.

In [38]:
## get country shape
countries = ee.FeatureCollection('WM/geoLab/geoBoundaries/600/ADM0')
ch = countries.filterMetadata('shapeName', 'equals', 'Switzerland')

## run prediction for Switzerland
flooded_prob = predict(roi=ch.geometry(), classifier=classifier)

## update the map
flooded_prob_vis = {'min':0, 'max':1, 'palette':cm.palettes.cividis}
Map.addLayer(flooded_prob.selfMask(), flooded_prob_vis, 'Flooded probability')
Map.add_colorbar(flooded_prob_vis, label="CH flooded probability", layer_name="CH flooded probability", font_size=9)
Map

Map(bottom=3879.0, center=[22.14670778001263, -66.01762052261185], controls=(WidgetControl(options=['position'…

In [41]:
## create export task
task = ee.batch.Export.image.toAsset(
  flooded_prob,
  description='flood475-ch-prediction',
  assetId=f"projects/{PROJECT_ID}/assets/flood475_ch_prediction",
  scale=30,
  maxPixels=500_000_000
)

## run and monitor the task
task.start()
while task.active():
  ts = task.status()
  if ts['start_timestamp_ms']>0:
    s = round((ts['update_timestamp_ms']-ts['start_timestamp_ms'])/1000)
  else:
    s = round((ts['update_timestamp_ms']-ts['creation_timestamp_ms'])/1000)
  print(f"task '{ts['description']}' is {ts['state']} for {s} seconds")
  time.sleep(60)
task.status()

task 'flood475-ch-prediction' is RUNNING for 0 seconds
task 'flood475-ch-prediction' is RUNNING for 60 seconds
task 'flood475-ch-prediction' is RUNNING for 120 seconds
task 'flood475-ch-prediction' is RUNNING for 180 seconds
task 'flood475-ch-prediction' is RUNNING for 240 seconds
task 'flood475-ch-prediction' is RUNNING for 300 seconds
task 'flood475-ch-prediction' is RUNNING for 360 seconds
task 'flood475-ch-prediction' is RUNNING for 420 seconds
task 'flood475-ch-prediction' is RUNNING for 480 seconds
task 'flood475-ch-prediction' is RUNNING for 540 seconds
task 'flood475-ch-prediction' is RUNNING for 600 seconds
task 'flood475-ch-prediction' is RUNNING for 660 seconds
task 'flood475-ch-prediction' is RUNNING for 720 seconds
task 'flood475-ch-prediction' is RUNNING for 780 seconds
task 'flood475-ch-prediction' is RUNNING for 840 seconds
task 'flood475-ch-prediction' is RUNNING for 900 seconds
task 'flood475-ch-prediction' is RUNNING for 960 seconds
task 'flood475-ch-prediction' is R